In [19]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Step 1: Load the data
df = pd.read_csv('augmented_train.csv')

In [20]:
# Step 2: Preprocess the data
# Separate features and target
df = df.drop(columns=['Id'])

features = df.drop(columns=['Pawpularity'])
target = df['Pawpularity']

features = features.select_dtypes(include=[np.number])

features = features.astype(np.float32)
target = target.astype(np.float32)

# Split into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

# Ensure the labels are in the correct shape
y_train = y_train.values.reshape(-1, 1)
y_val = y_val.values.reshape(-1, 1)



In [21]:
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(1, activation='linear')  # Output layer for regression
])

In [23]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='mse', 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Step 4: Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Step 5: Evaluate the model
val_predictions = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
print(f"Validation RMSE: {rmse:.4f}")

Epoch 1/100
1239/1239 [==============================] - 2s 1ms/step - loss: 0.2601 - root_mean_squared_error: 0.5100 - val_loss: 0.0430 - val_root_mean_squared_error: 0.2074 - lr: 0.0010
Epoch 2/100
1239/1239 [==============================] - 1s 1ms/step - loss: 0.0450 - root_mean_squared_error: 0.2122 - val_loss: 0.0423 - val_root_mean_squared_error: 0.2057 - lr: 0.0010
Epoch 3/100
1239/1239 [==============================] - 1s 1ms/step - loss: 0.0428 - root_mean_squared_error: 0.2068 - val_loss: 0.0428 - val_root_mean_squared_error: 0.2068 - lr: 0.0010
Epoch 4/100
1239/1239 [==============================] - 1s 1ms/step - loss: 0.0428 - root_mean_squared_error: 0.2069 - val_loss: 0.0425 - val_root_mean_squared_error: 0.2061 - lr: 0.0010
Epoch 5/100
1239/1239 [==============================] - 1s 1ms/step - loss: 0.0429 - root_mean_squared_error: 0.2072 - val_loss: 0.0425 - val_root_mean_squared_error: 0.2062 - lr: 0.0010
Epoch 6/100
1239/1239 [==============================] - 1s 